In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Cargar el dataset
file_path = './forestfires.csv'
df = pd.read_csv(file_path)

# Funciones de categorización
def categorize_ffmc(value):
    if value <= 30:
        return 'Bajo'
    elif value <= 60:
        return 'Moderado'
    elif value <= 80:
        return 'Alto'
    else:
        return 'Muy Alto'

def categorize_dmc(value):
    if value <= 10:
        return 'Bajo'
    elif value <= 20:
        return 'Moderado'
    elif value <= 30:
        return 'Alto'
    else:
        return 'Muy Alto'

def categorize_dc(value):
    if value <= 100:
        return 'Bajo'
    elif value <= 200:
        return 'Moderado'
    elif value <= 300:
        return 'Alto'
    else:
        return 'Muy Alto'

def categorize_isi(value):
    if value <= 3:
        return 'Bajo'
    elif value <= 6:
        return 'Moderado'
    elif value <= 12:
        return 'Alto'
    else:
        return 'Muy Alto'

def area_cat(area):
    if area == 0.0:
        return "No damage"
    elif area <= 1:
        return "low"
    elif area <= 25:
        return "moderate"
    elif area <= 100:
        return "high"
    else:
        return "very high"

# Aplicar las funciones de categorización
df['FFMC_category'] = df['FFMC'].apply(categorize_ffmc)
df['DMC_category'] = df['DMC'].apply(categorize_dmc)
df['DC_category'] = df['DC'].apply(categorize_dc)
df['ISI_category'] = df['ISI'].apply(categorize_isi)
df['area_category'] = df['area'].apply(area_cat)

# Seleccionar características y etiqueta
numeric_features = ['X', 'Y', 'temp', 'RH', 'wind', 'rain']
categorical_features = ['day', 'FFMC_category', 'DMC_category', 'DC_category', 'ISI_category']
X = df[numeric_features + categorical_features]
y = df['area_category']

# Preprocesamiento: Estandarizar características numéricas y codificar categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Transformar X
X_processed = preprocessor.fit_transform(X)

# Codificar la variable objetivo
y_encoded = pd.get_dummies(y).values  # Convertir a one-hot encoding

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)

# Construcción del modelo de red neuronal
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Salida con softmax para clasificación multiclase

# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Evaluar el rendimiento en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

# Generar reporte de clasificación
y_pred = model.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)
print(classification_report(y_test_labels, y_pred_labels, target_names=df['area_category'].unique()))

# Guardar el modelo entrenado
model.save('area_category_nn_model.h5')

# Cargar el modelo y hacer una predicción con nuevos datos
loaded_model = tf.keras.models.load_model('area_category_nn_model.h5')

# Ejemplo de nuevos datos
new_data = pd.DataFrame({
    'X': [7],
    'Y': [5],
    'temp': [22.0],
    'RH': [45],
    'wind': [3.1],
    'rain': [0.0],
    'day': ['fri'],
    'FFMC_category': ['Muy Alto'],
    'DMC_category': ['Alto'],
    'DC_category': ['Muy Alto'],
    'ISI_category': ['Moderado']
})

# Preprocesar nuevos datos
new_data_processed = preprocessor.transform(new_data)

# Realizar la predicción
prediction = loaded_model.predict(new_data_processed)
predicted_category = np.argmax(prediction, axis=1)
category_labels = df['area_category'].unique()
print(f"Predicción de la categoría del área quemada: {category_labels[predicted_category][0]}")


Epoch 1/100


/home/mcabre/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724423964.938349   13095 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 16:39:24.939774: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


13/13 - 1s - 58ms/step - accuracy: 0.1598 - loss: 1.7079 - val_accuracy: 0.3558 - val_loss: 1.5208
Epoch 2/100
13/13 - 0s - 4ms/step - accuracy: 0.3293 - loss: 1.5005 - val_accuracy: 0.3558 - val_loss: 1.3986
Epoch 3/100
13/13 - 0s - 4ms/step - accuracy: 0.4092 - loss: 1.3779 - val_accuracy: 0.4038 - val_loss: 1.3150
Epoch 4/100
13/13 - 0s - 4ms/step - accuracy: 0.3898 - loss: 1.3291 - val_accuracy: 0.4038 - val_loss: 1.2582
Epoch 5/100
13/13 - 0s - 4ms/step - accuracy: 0.4407 - loss: 1.2762 - val_accuracy: 0.4519 - val_loss: 1.2182
Epoch 6/100
13/13 - 0s - 4ms/step - accuracy: 0.4891 - loss: 1.2096 - val_accuracy: 0.4231 - val_loss: 1.1947
Epoch 7/100
13/13 - 0s - 4ms/step - accuracy: 0.4625 - loss: 1.2037 - val_accuracy: 0.4231 - val_loss: 1.1835
Epoch 8/100
13/13 - 0s - 4ms/step - accuracy: 0.4794 - loss: 1.2167 - val_accuracy: 0.4327 - val_loss: 1.1809
Epoch 9/100
13/13 - 0s - 4ms/step - accuracy: 0.4891 - loss: 1.1927 - val_accuracy: 0.4519 - val_loss: 1.1810
Epoch 10/100
13/13 - 

Epoch 75/100
13/13 - 0s - 4ms/step - accuracy: 0.6126 - loss: 0.9448 - val_accuracy: 0.4615 - val_loss: 1.2720
Epoch 76/100
13/13 - 0s - 3ms/step - accuracy: 0.5860 - loss: 0.9534 - val_accuracy: 0.4712 - val_loss: 1.2796
Epoch 77/100
13/13 - 0s - 4ms/step - accuracy: 0.5932 - loss: 0.9349 - val_accuracy: 0.4712 - val_loss: 1.2948
Epoch 78/100
13/13 - 0s - 3ms/step - accuracy: 0.6005 - loss: 0.9359 - val_accuracy: 0.4615 - val_loss: 1.2936
Epoch 79/100
13/13 - 0s - 3ms/step - accuracy: 0.6271 - loss: 0.9253 - val_accuracy: 0.4519 - val_loss: 1.3012
Epoch 80/100
13/13 - 0s - 3ms/step - accuracy: 0.6126 - loss: 0.9433 - val_accuracy: 0.4712 - val_loss: 1.2992
Epoch 81/100
13/13 - 0s - 4ms/step - accuracy: 0.5738 - loss: 0.9062 - val_accuracy: 0.4712 - val_loss: 1.3124
Epoch 82/100
13/13 - 0s - 3ms/step - accuracy: 0.5811 - loss: 0.9411 - val_accuracy: 0.4423 - val_loss: 1.3053
Epoch 83/100
13/13 - 0s - 3ms/step - accuracy: 0.5956 - loss: 0.9221 - val_accuracy: 0.4423 - val_loss: 1.3090
E

/home/mcabre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcabre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcabre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

   No damage       0.51      0.69      0.59        51
         low       0.00      0.00      0.00         9
    moderate       0.00      0.00      0.00         5
        high       0.37      0.35      0.36        37
   very high       0.00      0.00      0.00         2

    accuracy                           0.46       104
   macro avg       0.18      0.21      0.19       104
weighted avg       0.38      0.46      0.42       104



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicción de la categoría del área quemada: high
